In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import chardet  

 
def detectar_arquivo():
    try:
        
        with open('taxa_de_desistencia.csv', 'rb') as f:
            result = chardet.detect(f.read(10000))
        
        #ler diferentes configurações
        for sep in [';', ',', '\t']:
            try:
                df = pd.read_csv('taxa_de_desistencia.csv', 
                               sep=sep, 
                               encoding=result['encoding'],
                               engine='python')
                if len(df.columns) > 1:
                    print(f"Arquivo lido com sucesso! Separador: '{sep}'")
                    return df
            except:
                continue
        
        try:
            df = pd.read_csv('taxa_de_desistencia.csv', 
                           encoding=result['encoding'])
            print("Arquivo lido (separador automático)")
            return df
        except:
            print("Falha ao ler o arquivo. Mostrando diagnóstico...")
            with open('taxa_de_desistencia.csv', 'r', encoding=result['encoding']) as f:
                print("\nPrimeiras 5 linhas CRUS do arquivo:")
                for _ in range(5):
                    print(f.readline().strip())
            return None

    except Exception as e:
        print(f"Erro crítico: {e}")
        return None

def mostrar_estrutura(df):
    print("\n Estrutura do arquivo:")
    print(f"- Total de linhas: {len(df)}")
    print(f"- Colunas detectadas: {list(df.columns)}")
    
    print("\n Amostra de dados:")
    print(df.head(3).to_string(index=False))
    
    print("\n Tipos de dados:")
    print(df.dtypes)

def filtrar_dados(df):
    # coluna de anos (que contenha números entre 2013-2023)
    ano_col = None
    for col in df.columns:
       
        numeric_series = pd.to_numeric(df[col], errors='coerce')
        
        if any(numeric_series.dropna().between(2013, 2023)):
            ano_col = col
            
            df[ano_col] = numeric_series
            print(f" Coluna de ano detectada: '{ano_col}'")
            break
    
    if not ano_col:
        print(" Nenhuma coluna de ano detectada. Pulando filtro de anos.")
        return df
    
    
    original_rows = len(df)
    df = df[df[ano_col].notna() & df[ano_col].between(2013, 2023)]
    filtered_rows = len(df)
    print(f"\n Filtro de anos (2013-2023):")
    print(f"- Linhas removidas: {original_rows - filtered_rows}")
    print(f"- Linhas restantes: {filtered_rows}")
    
    
    colunas_para_remover = []
    
    
    for col in df.columns:
        if '6.filtro' in col.lower() or '6.filtro' in col.lower().replace(' ', ''):
            colunas_para_remover.append(col)
    
    
    for col in df.columns:
        if '8.símbolo' in col.lower() or '8.simbolo' in col.lower() or '8.símbolo' in col.lower().replace(' ', ''):
            colunas_para_remover.append(col)
    
    if colunas_para_remover:
        print(f"\n Removendo colunas indesejadas: {colunas_para_remover}")
        df = df.drop(columns=colunas_para_remover, errors='ignore')
        print(f"Colunas restantes: {list(df.columns)}")
    
    return df


def criar_heatmap(df):
    
    ano_col = None
    for col in df.columns:
        numeric_series = pd.to_numeric(df[col], errors='coerce')
        if any(numeric_series.dropna().between(2013, 2023)):
            ano_col = col
            df[ano_col] = numeric_series
            print(f" Coluna de ano detectada: '{ano_col}'")
            break
    
    if not ano_col:
        print("Nenhuma coluna de ano detectada. Usando a primeira coluna como fallback.")
        ano_col = df.columns[0]  
        df[ano_col] = pd.to_numeric(df[ano_col], errors='coerce')

    valor_col = None
    for col in df.columns:
        if pd.api.types.is_numeric_dtype(df[col]):
            valor_col = col
            print(f" Coluna de valores detectada: '{valor_col}'")
            break
    
    if not valor_col:
        print(" Nenhuma coluna numérica encontrada para valores!")
        return
    
    curso_col = None
    for col in df.columns:
        if 'curso' in col.lower() or df[col].dtype == 'object':
            curso_col = col
            print(f" Coluna de curso/categoria detectada: '{curso_col}'")
            break
    
    if not curso_col:
        df['Curso_Generico'] = 'Dados'  
        curso_col = 'Curso_Generico'
        print("Nenhuma coluna de curso detectada. Criando coluna genérica 'Curso_Generico'.")
    
    print("\n--- Debugging before pivot_table ---")
    print(f"Usando '{curso_col}' como index.")
    print(f"Usando '{ano_col}' como columns.")
    print(f"Usando '{valor_col}' como values.")
    print(f"Tipo de dado da coluna '{ano_col}': {df[ano_col].dtype}")
    print(f"Primeiros valores da coluna '{ano_col}':")
    print(df[ano_col].head().to_string(index=False))
    print("--- End Debugging ---")

print(" Analisando seu arquivo...")
dados = detectar_arquivo()

if dados is not None:
    mostrar_estrutura(dados)
    
    dados_filtrados = filtrar_dados(dados)
    
    print("\n Estrutura após filtros:")
    mostrar_estrutura(dados_filtrados)
    
    criar_heatmap(dados_filtrados)
else:
    print("\n Dicas para resolver:")
    print("1. Verifique se o arquivo está na mesma pasta do script")
    print("2. O arquivo deve ter extensão .csv")
    print("3. Envie as 5 primeiras linhas do arquivo para que eu possa ajudar melhor")
    

 Analisando seu arquivo...
Arquivo lido com sucesso! Separador: ','

 Estrutura do arquivo:
- Total de linhas: 55791
- Colunas detectadas: ['1. Ano', '2. Nome Região (Portugal)', '3. Âmbito Geográfico', '4. Filtro 1', '5. Filtro 2', '6. Filtro 3', '7. Escala', '8. Símbolo', '9. Valor']

 Amostra de dados:
 1. Ano 2. Nome Região (Portugal) 3. Âmbito Geográfico   4. Filtro 1 5. Filtro 2  6. Filtro 3    7. Escala 8. Símbolo  9. Valor
 1995.0                  Portugal                  NaN Cursos Gerais     10º ano          NaN % dos alunos        NaN      28.6
 1995.0                  Portugal                  NaN Cursos Gerais     11º ano          NaN % dos alunos        NaN       9.6
 1995.0                  Portugal                  NaN Cursos Gerais     12º ano          NaN % dos alunos        NaN      30.1

 Tipos de dados:
1. Ano                       float64
2. Nome Região (Portugal)     object
3. Âmbito Geográfico          object
4. Filtro 1                   object
5. Filtro 2    

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=bd689bad-aedf-4263-9c72-b392a6c7f0cf' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>